In [25]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times
import io # Input/Output Bytes objects
import time # For sleep in for loop

# Analysis

import numpy as np
import pandas as pd
import geopandas as gpd

# Visualization

import contextily as ctx # Basemaps
import matplotlib.pyplot as plt # Basic Plotting
import seaborn as sns # Statistical Plotting

import matplotlib.cm as cm
import matplotlib.colors as mcolors


# Definitions

In [2]:
# This is my personal API key... Please use responsibly!

api = input('Please enter your Purple Air api key')

Please enter your Purple Air api key 51592903-B445-11ED-B6F4-42010A800007


In [3]:
# Get Sensor Locations

def getSensorsData(query='', api_read_key=''):

    # my_url is assigned the URL we are going to send our request to.
    url = 'https://api.purpleair.com/v1/sensors?' + query
    
    # print('Here is the full url for the API call:\n\n', url)

    # my_headers is assigned the context of our request we want to make. In this case
    # we will pass through our API read key using the variable created above.
    my_headers = {'X-API-Key':api_read_key}

    # This line creates and sends the request and then assigns its response to the
    # variable, r.
    response = requests.get(url, headers=my_headers)

    # We then return the response we received.
    return response

# Data

In [4]:
## Load Data

data_path = os.path.join('..', '..', 'data')

spikes = pd.read_csv(os.path.join(data_path, 'all_spikes_06152022-04202023.csv'))

spikes['timestamp'] = pd.to_datetime(spikes.timestamp)

summaries = pd.read_csv(os.path.join(data_path, 'daily_summaries_06152022-04202023.csv'))
summaries['date'] = pd.to_datetime(summaries.date)

summaries_no_spikes = pd.read_csv(os.path.join(data_path, 'daily_summaries_06152022-04202023.csv'))
summaries_no_spikes['date'] = pd.to_datetime(summaries_no_spikes.date)

# Sensor Indices (from City of Minneapolis)

indices_path = os.path.join(data_path, 'PA IDs and indexes.xlsx')

sensor_info = pd.read_excel(indices_path) # Load as DataFrame

sensor_ids = sensor_info['Sensor Index'].dropna().astype(int)

In [5]:
# Locations

sensor_string = 'show_only=' + '%2C'.join(sensor_ids.astype(str))

query = 'fields=latitude%2Clongitude&' + sensor_string

response = getSensorsData(query, api)

response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

sensors_df = pd.DataFrame(data, columns = col_names)

In [6]:
# Spatialize

sensors = gpd.GeoDataFrame(sensors_df.sensor_index,
                 geometry = gpd.points_from_xy(x=sensors_df.longitude, y=sensors_df.latitude),
                           crs = 'EPSG:4326'
                ).to_crs('EPSG:26915')

# Spikes Movie

In [7]:
spikes.head()

,sensor_index,timestamp,pm25
0,143226,2022-06-17 22:10:00,58.910
1,143226,2022-06-17 22:20:00,35.495
2,143226,2022-06-17 22:00:00,319.660
3,142720,2022-06-18 03:20:00,33.595
4,142720,2022-06-18 03:30:00,36.026


In [8]:
days = spikes.timestamp.apply(lambda x: x.date())

In [9]:
days.value_counts()

2023-01-11    6072
2023-01-10    6028
2023-01-09    5882
2023-01-08    5696
2022-11-23    5674
              ... 
2022-06-22       1
2022-07-22       1
2023-04-16       1
2022-10-17       1
2022-08-08       1
Name: timestamp, Length: 303, dtype: int64

In [52]:
# mintime = spikes.timestamp.min()
# maxtime = spikes.timestamp.max()

mintime = dt.datetime(2023,1,7)
maxtime = dt.datetime(2023,1,10)

In [53]:
timestamp_range = pd.date_range(start = mintime,
                           end = maxtime,
                           freq = '10min'
                          )

In [54]:
timestamp_range

DatetimeIndex(['2023-01-07 00:00:00', '2023-01-07 00:10:00',
               '2023-01-07 00:20:00', '2023-01-07 00:30:00',
               '2023-01-07 00:40:00', '2023-01-07 00:50:00',
               '2023-01-07 01:00:00', '2023-01-07 01:10:00',
               '2023-01-07 01:20:00', '2023-01-07 01:30:00',
               ...
               '2023-01-09 22:30:00', '2023-01-09 22:40:00',
               '2023-01-09 22:50:00', '2023-01-09 23:00:00',
               '2023-01-09 23:10:00', '2023-01-09 23:20:00',
               '2023-01-09 23:30:00', '2023-01-09 23:40:00',
               '2023-01-09 23:50:00', '2023-01-10 00:00:00'],
              dtype='datetime64[ns]', length=433, freq='10T')

In [55]:
xmin, ymin, xmax, ymax = sensors.total_bounds

In [56]:
import warnings

warnings.filterwarnings('ignore')

In [84]:
for timestamp in timestamp_range:
    
    f, ax = plt.subplots(1, figsize = (6,8))
    
    title = 'Spikes - ' + str(timestamp.date())
    ax.set_title(title,
              fontdict={'fontsize': '25',
                         'fontweight' : 'semibold'}
            )
    
    # Add time annotation
    
    ax.annotate(str(timestamp.time()),
                (.7,.95),
               xycoords = 'axes fraction',
               fontweight = 'semibold',
               fontsize = 16,
               backgroundcolor = 'white')
    
    # Select the data & Plot
    
    select = spikes[spikes.timestamp == timestamp]
    
    select_gdf = sensors.merge(select, left_on = 'sensor_index', right_on = 'sensor_index')
    
    select_gdf.plot('pm25',
                    # edgecolor = 'Grey',
                    cmap = 'Reds',
                    vmin = 25,
                    vmax = 100,
                  linewidth = .5,
                  markersize = 100,
                    marker = '^',
                 ax = ax)
    
    ax.set_ylim(ymin, ymax)
    ax.set_xlim(xmin, xmax)
    
    # Make Legend
    
    # define a mappable based on which the colorbar will be drawn
    mappable = cm.ScalarMappable(
        norm=mcolors.Normalize(25, 100),
        cmap='Reds'
    )

    # define position and extent of colorbar
    cb_ax = f.add_axes([.85, 0.1, .03, .8])

    # draw colorbar
    cbar = f.colorbar(mappable, cax=cb_ax, orientation='vertical')
    
    cb_ax.set_title('PM2.5 Reading', rotation = 270, fontweight = 'semibold',x=3.5,y=0.5)

    # Basemap
    
    ctx.add_basemap(ax=ax,
                crs = select_gdf.crs,
               source = ctx.providers.OpenStreetMap.Mapnik)
    
    # Wrap Up
    
    ax.set_axis_off()
    f.tight_layout()
    
    savename = 'Spikes_' + str(timestamp)
    
    savename = savename.replace(' ','_')
    
#     plt.savefig('SpikeFrames/'+savename+'.jpg')
#     plt.show()
    
    
#     break
    
    
    plt.savefig('SpikeFrames/'+savename+'.jpg')
    
    plt.close()

In [ ]:
# Then you're going to need to use imagemagick in the command line to make the movie

# cd to images folder
# get images as a list
# ims=$(ls)
# Convert to a Gif with .25 second frames
# convert -delay 25 -dispose previous $ims ../one_iteration.gif
# Loop the gif with itself with 5 second delay
# convert ../one_iteration.gif \( +clone -set delay 500 \) +swap +delete ../full_movie.gif
